In [1]:
import satellite_images_nso.api.nso_georegion as nso
from satellite_images_nso._nso_data_extraction import nso_api
import rasterio
import numpy as np
from matplotlib import pyplot as plt
from rasterio.plot import show
import glob

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.4.3-CAPI-1.8.3 r4285) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# Search for satellite images in geojson and download links

In [2]:
# give path to geojson en output,!!!!! rewrite these directories to your file system!!!!
path_geojson = "C:/Users/pzhadmin/Documents/natura2000_coepelduynen.geojson"
output_path = "E:/data/coepelduynen/"

In [3]:
nso_username = 'micwin'
nso_password = 'REDACTED'

# This method fetches all the download links to all the satelliet images which contain region in the geojson
georegion = nso.nso_georegion(path_geojson,output_path,\
***REMOVED***nso_username,\
                            nso_password)

In [4]:
links = georegion.retrieve_download_links( max_diff = 0.9)

Cloudcover check:
Passed cloud check
Going into region check:
Max_diff in method 0.9
[[ 4.43428239  4.43436894  4.43436895 ...  4.43388158  4.4339625
   4.43428239]
 [52.23760322 52.23737775 52.23737772 ... 52.23755883 52.23755515
  52.23760322]]
[[ 4.43428239  4.43436894  4.43436895 ...  4.43388158  4.4339625
   4.43428239]
 [52.23760322 52.23737775 52.23737772 ... 52.23755883 52.23755515
  52.23760322]]
Passed region check
Cloudcover check:
Passed cloud check
Going into region check:
Max_diff in method 0.9
[[ 4.43428239  4.43436894  4.43436895 ...  4.43388158  4.4339625
   4.43428239]
 [52.23760322 52.23737775 52.23737772 ... 52.23755883 52.23755515
  52.23760322]]
[[ 4.43428239  4.43436894  4.43436895 ...  4.43388158  4.4339625
   4.43428239]
 [52.23760322 52.23737775 52.23737772 ... 52.23755883 52.23755515
  52.23760322]]
Passed region check
Cloudcover check:
Passed cloud check
Going into region check:
Max_diff in method 0.9
[[ 4.43428239  4.43436894  4.43436895 ...  4.43388158  4.

KeyboardInterrupt: 

# Download, Crop and calculate NDVI for a LandSat Image

### For a specific image

In [ ]:
# Downloads a satellite image from the NSO, makes a crop out of it so it fits the geojson region and calculates the NVDI index.
# The output will stored in the designated output folder.

image = links[0]
georegion.execute_link(image)

# The parameters are : execute_link(self, link, delete_zip_file = False,delete_source_files = True, 
# check_if_file_exists = True)

### For a group of images 

In [ ]:
done_files = []
# Filter out already done links.
for file in glob.glob(output_path+"/*.tif"):
    print(file.split("\\")[1].split("-")[0])
    done_files.append(file.split("\\")[1].split("-")[0])

In [ ]:
x_sat = 1
for link in links:
    if 'SV' in link and '50cm' in link and "RGBI" in link and "SV1-01" in link:
        print(link)
        x_sat = x_sat+1
        print(x_sat)

In [ ]:
links

In [ ]:
# Loop through and download all 50 cm resolution SuperView Images.
SV_links = []
for link in links:
    if 'SV' in link and '50cm' in link and "RGBI" in link:
        check_list = [link.find(file) for file in done_files]
        if sum(1 for number in check_list if number > 0) == 0:
            print(link)
            SV_links.append(link)

In [ ]:
for link in SV_links:
    print(link)
    georegion.execute_link(link, calculate_nvdi = False)

In [ ]:
# loop through and download all the 200cm resolution SuperView images
for link in links:
    if 'SV' in link and '200cm' in link and "RGBI" in link:
        
            print(link)
            try:
                georegion.execute_link(link, calculate_nvdi = False)
            except Exception as e:
                print(e)

In [ ]:
for file in glob.glob("C:/repos/satellite-images-nso/output/*_heel_geojson_perceel_cropped.tif"):
    print(file)
    
    src = rasterio.open(file)
    plot_out_image = np.clip(src.read()[2::-1],
                    0,2200)/2200 # out_image[2::-1] selects the first three items, reversed

    plt.figure(figsize=(10,10))
    rasterio.plot.show(plot_out_image,
          transform=src.transform)

In [ ]:
# loop through and download all the 200cm Trisat images
for link in links:
    if 'Tri' in link and "80cm" in link and "8bit" in link:
        try:
            georegion.execute_link(link)
        except Exception as e:
            print(e)

# Normalize LandSat image.

### Multi date relative normalisation.

In [ ]:
# Imports the normalisation library.
from satellite_images_nso.__normalisation import normalisation

# Path to a Superview satellite image.
path = "20190401_111723_SV1-04_200cm_RD_11bit_RGBI_DenHaag_solleveld_sweco_cropped.tif"


normalisation.multidate_normalisation_75th_percentile(path)


### Black spot normalisation.

In [ ]:
from satellite_images_nso.__normalisation import normalisation

In [ ]:
path = "C:/repos/github/satellite_images_nso/data_satellite/20210302_111341_SV1-03_SV_RD_11bit_RGBI_200cm_Brouwersdam_Grevelingen_21_cropped.tif"

In [ ]:
normalisation.multi_date_dark_spot_normalisation(path,"20210302")

# Unpack multipolygon

In [ ]:
import geopandas as gpd

In [ ]:
path = "C:/repos/satellite-images-nso/input_data/enkele percelen_MD/enkele percelen MD.shp"

In [ ]:
gdf = gpd.read_file(path)

In [ ]:
gdf = gdf.to_crs("EPSG:4326")

In [ ]:
for x in range(0,len(gdf)-1):
    gdf[x:x+1].to_file("C:/repos/satellite-images-nso/input_data/enkele percelen_MD/geojson/"+str(x)+"_perceel.json", driver="GeoJSON")

In [ ]:
gdf = gpd.read_file("C:/repos/satellite-images-nso/input_data/enkele_percelen_MD/heel_geojson_perceel.geojson")

In [ ]:
gdf = gdf.explode()

In [ ]:
gdf.to_file("C:/repos/satellite-images-nso/input_data/enkele_percelen_MD/heel_geojson_perceel.geojson", driver='GeoJSON')

# Do multiploygons

In [ ]:
import glob

In [ ]:
for file in glob.glob("C:/repos/satellite-images-nso/input_data/enkele percelen_MD/geojson/*"):
    print(file)
    path_geojson = file
    output_path = "C:/repos/satellite-images-nso/output"
    nso_username = 'micwin'
    nso_password = 'REDACTED'

    # This method fetches all the download links to all the satelliet images which contain region in the geojson
    georegion = nso.nso_georegion(path_geojson,output_path,\
***REMOVED***    nso_username,\
***REMOVED***  nso_password)

    links = georegion.retrieve_download_links()
    for link in links:
        if 'SV' in link and '50cm' in link and "RGBI" in link:
            georegion.execute_link(link)

# Get multiple polygons out of wider region

In [ ]:
import satellite_images_nso.api.sat_manipulator as sat_manipulator

In [ ]:
from satellite_images_nso.miscellaneous import miscellaneous

In [ ]:
for ageojson in glob.glob("C:/repos/satellite-images-nso/input_data/enkele_percelen_MD/geojson/*"):
    print(ageojson)
    for atiffile in glob.glob("C:/repos/satellite-images-nso/output/*_heel_geojson_perceel_cropped.tif"):
        print(atiffile)
        try:
            sat_manipulator.crop_nso_satellite_image(ageojson,atiffile,atiffile.replace(".tif","_"+ageojson.split("/")[-1].replace("\\","_")+".tif"))
        except:
            print("Error")

In [ ]:
for atiffile in glob.glob("C:/repos/satellite-images-nso/output/*_heel_geojson_perceel_cropped_geojson_0_perceel.json.tif"):
    print(atiffile)
    
    src = rasterio.open(atiffile)
    plot_out_image = np.clip(src.read()[2::-1],
                    0,2200)/2200 # out_image[2::-1] selects the first three items, reversed

    plt.figure(figsize=(10,10))
    rasterio.plot.show(plot_out_image,
          transform=src.transform)

## Merge back into one polygon

In [ ]:
# Loop through and download all 50 cm resolution SuperView Images.
for link in links:
    if 'SV' in link and '200cm' in link and "RGBI" in link:
        try:
            print("---------------------")
            print(link.split("/")[-1])
            tif_list = glob.glob("C:/repos/satellite-images-nso/output/"+link.split("/")[-1]+"_200cm*_perceel.json.tif")
            print(tif_list)
            miscellaneous.merge_tif_files(tif_list , "C:/repos/satellite-images-nso/output/"+str(link.split("/")[-1])+"_percelen.tif")
        except Exception as e: 
            print(str(e))

# Other functionalities

In [ ]:
# The sat_manipulator gives other handy transmations on satellite data.
import satellite_images_nso.api.sat_manipulator as sat_manipulator

In [2]:
file = "C:/repos/satellite-images-nso/output/20210921_111555_SV1-04_SV_RD_11bit_RGBI_200cm_Rotterdam_heel_geojson_perceel_cropped_geojson_0_perceel.json.tif"

In [3]:
current_df = sat_manipulator.tranform_vector_to_pixel_gpdf("C:/repos/satellite-images-nso/output/20210921_111555_SV1-04_SV_RD_11bit_RGBI_200cm_Rotterdam_heel_geojson_perceel_cropped_geojson_0_perceel.json.tif" , add_ndvi_column = True)

NameError: name 'sat_manipulator' is not defined

In [ ]:
current_df

In [ ]:
current_df[['blue','green','red','nir','nvdi',"X","Y", 'geometry']]

In [ ]:
# This function reads a .tif file, which is a format the satellite data is stored in,  and converts it to a pixel based geopandas dataframe.
# For machine learning purposes.
path_to_vector = "path/to/folder/*.tif"
geo_df_pixel = sat_manipulator.tranform_vector_to_pixel_gpdf(path_to_vector)

In [ ]:
# The sat_manipulator gives other handy transmations on satellite data.
import satellite_images_nso._manipulation.nso_manipulator as nso_manipulator

In [ ]:
path_geojson = "C:/repos/github/satellite_images_nso/input_data/waterleidingduin_aaneensluitende_polygon.geojson"
path_to_tif = "C:/repos/github/satellite_images_nso/data_satellite/20190422_111333_SV1-01_50cm_RD_8bit_RGB_Noordwijkerhout/20190422_111333_SV1-01_50cm_RD_8bit_RGB_Noordwijkerhout.tif"
path_to_tif_cropped = "C:/repos/github/satellite_images_nso/data_satellite/20190422_111333_SV1-01_50cm_RD_8bit_RGB_Noordwijkerhout/20190422_111333_SV1-01_50cm_RD_8bit_RGB_Noordwijkerhout_cropped.tif"

In [ ]:
nso_manipulator.__make_the_cut(path_geojson,path_to_tif, path_to_tif_cropped)

In [ ]:
nso_manipulator

In [ ]:
# Crop a .tif file.
georegion.crop_and_calculate_nvdi(path_to_tif,False)